In [2]:
import pandas as pd
import numpy as np
import nltk
import string

import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

NameError: name 'plt' is not defined

In [3]:
df = pd.read_csv("C:/Users/Ankan SInha/Desktop/Final/NON-Irony/users/e34c64a05273012b2868d8b04b812089.csv")
df

,Tweets
0,"#USER# #USER# But frankly Anna, you being ther..."
1,aND MY FRIEND KEEPS SAYING TO QUIT SCHOOL 😭😭😭 ...
2,#USER# Jacobin is on the same level of credibi...
3,#USER# We use Target brand Sensitive formula. ...
4,Russian billionaire Abramovich not contacted a...
5,#USER# Plutonium more dangerous than carbon. B...
6,#USER# 1975 in the DMZ. I was a guard at Panmu...
7,"#USER# ""Tch - calm down, I was only teasing......"
8,I real deal don’t give nobody none 😂 wtf. The ...
9,#USER# peace no war\r\nMizuho airen rose's hus...


In [4]:
for col in df.columns:
    print(col, df[col].isnull().sum())

Tweets 0


In [5]:
rws = df.loc[:, ['Tweets']]

In [6]:
rws['Tweets_token'] = rws['Tweets'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

,Tweets,Tweets_token
0,"#USER# #USER# But frankly Anna, you being ther...","[#USER#, #USER#, But, frankly, Anna,, you, bei..."
1,aND MY FRIEND KEEPS SAYING TO QUIT SCHOOL 😭😭😭 ...,"[aND, MY, FRIEND, KEEPS, SAYING, TO, QUIT, SCH..."
2,#USER# Jacobin is on the same level of credibi...,"[#USER#, Jacobin, is, on, the, same, level, of..."
3,#USER# We use Target brand Sensitive formula. ...,"[#USER#, We, use, Target, brand, Sensitive, fo..."
4,Russian billionaire Abramovich not contacted a...,"[Russian, billionaire, Abramovich, not, contac..."


In [7]:
rws['rating_description_str'] = [' '.join(map(str, l)) for l in rws['Tweets_token']]
rws['tokenized'] = rws['rating_description_str'].apply(word_tokenize)
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
stop_words = set(stopwords.words('english'))
new_stopwords = ["#", "url", "user",'a','an','the','of','or','is','it','for','have','has']
stopwrd = nltk.corpus.stopwords.words('english')
stopwrd.extend(new_stopwords)

rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in new_stopwords])
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
rws.head()

,Tweets,Tweets_token,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"#USER# #USER# But frankly Anna, you being ther...","[#USER#, #USER#, But, frankly, Anna,, you, bei...","#USER# #USER# But frankly Anna, you being ther...","[#, USER, #, #, USER, #, But, frankly, Anna, ,...","[#, user, #, #, user, #, but, frankly, anna, ,...","[user, user, but, frankly, anna, you, being, t...","[but, frankly, anna, you, being, there, no, gu...","[(but, CC), (frankly, RB), (anna, IN), (you, P...","[(but, n), (frankly, r), (anna, n), (you, n), ...","[but, frankly, anna, you, be, there, no, guara..."
1,aND MY FRIEND KEEPS SAYING TO QUIT SCHOOL 😭😭😭 ...,"[aND, MY, FRIEND, KEEPS, SAYING, TO, QUIT, SCH...",aND MY FRIEND KEEPS SAYING TO QUIT SCHOOL 😭😭😭 ...,"[aND, MY, FRIEND, KEEPS, SAYING, TO, QUIT, SCH...","[and, my, friend, keeps, saying, to, quit, sch...","[and, my, friend, keeps, saying, to, quit, sch...","[and, my, friend, keeps, saying, to, quit, sch...","[(and, CC), (my, PRP$), (friend, NN), (keeps, ...","[(and, n), (my, n), (friend, n), (keeps, v), (...","[and, my, friend, keep, say, to, quit, school,..."
2,#USER# Jacobin is on the same level of credibi...,"[#USER#, Jacobin, is, on, the, same, level, of...",#USER# Jacobin is on the same level of credibi...,"[#, USER, #, Jacobin, is, on, the, same, level...","[#, user, #, jacobin, is, on, the, same, level...","[user, jacobin, is, on, the, same, level, of, ...","[jacobin, on, same, level, credibility, as, br...","[(jacobin, NN), (on, IN), (same, JJ), (level, ...","[(jacobin, n), (on, n), (same, a), (level, n),...","[jacobin, on, same, level, credibility, a, bre..."
3,#USER# We use Target brand Sensitive formula. ...,"[#USER#, We, use, Target, brand, Sensitive, fo...",#USER# We use Target brand Sensitive formula. ...,"[#, USER, #, We, use, Target, brand, Sensitive...","[#, user, #, we, use, target, brand, sensitive...","[user, we, use, target, brand, sensitive, form...","[we, use, target, brand, sensitive, formula, t...","[(we, PRP), (use, VBP), (target, JJ), (brand, ...","[(we, n), (use, v), (target, a), (brand, n), (...","[we, use, target, brand, sensitive, formula, t..."
4,Russian billionaire Abramovich not contacted a...,"[Russian, billionaire, Abramovich, not, contac...",Russian billionaire Abramovich not contacted a...,"[Russian, billionaire, Abramovich, not, contac...","[russian, billionaire, abramovich, not, contac...","[russian, billionaire, abramovich, not, contac...","[russian, billionaire, abramovich, not, contac...","[(russian, JJ), (billionaire, NN), (abramovich...","[(russian, a), (billionaire, n), (abramovich, ...","[russian, billionaire, abramovich, not, contac..."


In [8]:
rws.drop('tokenized', inplace=True, axis=1)

In [9]:
rws.to_csv('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/User_3.csv')

In [10]:
file = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/lem3.csv', encoding="utf8")
read_data = file.read()
d_per_word = read_data.split()
print('Total Words:', len(d_per_word))

Total Words: 4941


In [11]:
File = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/User_3.csv',encoding="utf8") #open file
lines = File.read() #read all lines
print (lines)
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
v=list(nouns)
v.remove("'")
''.join(v)

,Tweets,Tweets_token,rating_description_str,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"#USER# #USER# But frankly Anna, you being there is no guarantee of anything if your assessment of Dardai is that he was lucky and/or completely reliant on his assistants.","['#USER#', '#USER#', 'But', 'frankly', 'Anna,', 'you', 'being', 'there', 'is', 'no', 'guarantee', 'of', 'anything', 'if', 'your', 'assessment', 'of', 'Dardai', 'is', 'that', 'he', 'was', 'lucky', 'and/or', 'completely', 'reliant', 'on', 'his', 'assistants.']","#USER# #USER# But frankly Anna, you being there is no guarantee of anything if your assessment of Dardai is that he was lucky and/or completely reliant on his assistants.","['#', 'user', '#', '#', 'user', '#', 'but', 'frankly', 'anna', ',', 'you', 'being', 'there', 'is', 'no', 'guarantee', 'of', 'anything', 'if', 'your', 'assessment', 'of', 'dardai', 'is', 'that', 'he', 'was', 'lucky', 'and/or', 'completely', 'reliant', 'on', 'his', 'assistants', '.']

ValueError: list.remove(x): x not in list

In [14]:
File = open('C:/Users/Ankan SInha/Desktop/Final/NON-Irony/user_processed/lem3.csv', encoding="utf8") #open file
lines = File.read() #read all lines
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
nouns.sort()

In [12]:
nounx = list(dict.fromkeys(nouns))
#print(nounx)
x=len(nounx)
print(x)

3260


In [13]:
from nltk.corpus import wordnet as wn
def get_some_word_synonyms(word):
    word = word.lower()
    synonyms = []
    synsets = wordnet.synsets(word[1:])
    print(synsets)
   
for word in nounx:
    #if (synsets == 2):
    #print((word[1:]))
    synsets = get_some_word_synonyms(word)
    #print (synonyms)
    #x = len(synsets)
    #print (x)


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[Synset('merely.r.01')]
[Synset('honestly.r.01')]
[]
[Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01')]
[Synset('no.n.01'), Synset('nobelium.n.01'), Synset('no.a.01'), Synset('no.r.01'), Synset('no.r.02'), Synset('no.r.03')]
[Synset('guarantee.n.01'), Synset('guarantee.n.02'), Synset('guarantee.n.03'), Synset('guarantee.v.01'), Synset('guarantee.v.02'), Synset('undertake.v.03'), Synset('guarantee.v.04')]
[]
[]
[Synset('appraisal.n.01'), Synset('assessment.n.02'), Synset('assessment.n.03'), Synset('judgment.n.02')]
[]
[Synset('helium.n.01'), Synset('he.n.02')]
[Synset('washington.n.02'), Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), 

[Synset('case_shot.n.01'), Synset('canister.n.02')]
[Synset('oregon.n.01'), Synset('operating_room.n.01')]
[Synset('two.n.01'), Synset('deuce.n.04'), Synset('two.s.01')]
[Synset('target.n.01'), Synset('prey.n.01'), Synset('target.n.03'), Synset('target.n.04'), Synset('aim.n.02'), Synset('target.v.01')]
[Synset('formula.n.01'), Synset('recipe.n.01'), Synset('formula.n.03'), Synset('formula.n.04'), Synset('convention.n.02'), Synset('formula.n.06'), Synset('rule.n.11')]
[Synset('information_technology.n.01')]
[Synset('cadmium.n.01'), Synset('candle.n.02'), Synset('certificate_of_deposit.n.01'), Synset('compact_disk.n.01'), Synset('four_hundred.s.01')]
[]
[]
[]
[]
[]
[]
[]
[Synset('not.r.01')]
[Synset('nationality.n.01'), Synset('nationality.n.02')]
[Synset('question.n.01'), Synset('inquiry.n.01'), Synset('inquiry.n.03')]
[]
[Synset('url.n.01')]
[Synset('contact.n.01'), Synset('contact.n.02'), Synset('contact.n.03'), Synset('contact.n.04'), Synset('contact.n.05'), Synset('liaison.n.02'), S

[Synset('arch.n.01'), Synset('arch.n.02'), Synset('arch.n.03'), Synset('arch.n.04'), Synset('arch.v.01'), Synset('arch.s.01'), Synset('arch.s.02'), Synset('arch.s.03')]
[Synset('milligram.n.01'), Synset('magnesium.n.01')]
[Synset('foot.n.02')]
[Synset('oat.n.01'), Synset('oat.n.02')]
[Synset('check.n.01'), Synset('assay.n.01'), Synset('check.n.03'), Synset('arrest.n.02'), Synset('confirmation.n.01'), Synset('check.n.06'), Synset('check_mark.n.01'), Synset('hindrance.n.01'), Synset('check.n.09'), Synset('check.n.10'), Synset('bridle.n.02'), Synset('check.n.12'), Synset('check.n.13'), Synset('check.v.01'), Synset('check.v.02'), Synset('see.v.10'), Synset('control.v.02'), Synset('check.v.05'), Synset('check.v.06'), Synset('check.v.07'), Synset('check.v.08'), Synset('match.v.01'), Synset('check.v.10'), Synset('discipline.v.01'), Synset('check.v.12'), Synset('check.v.13'), Synset('check.v.14'), Synset('check.v.15'), Synset('check.v.16'), Synset('check.v.17'), Synset('check.v.18'), Synset('c

[Synset('decelerate.v.01'), Synset('slow.v.02'), Synset('slow.v.03'), Synset('slow.a.01'), Synset('slow.a.02'), Synset('dense.s.04'), Synset('slow.a.04'), Synset('boring.s.01'), Synset('dull.s.08'), Synset('slowly.r.01'), Synset('behind.r.03')]
[Synset('pace.n.01'), Synset('footstep.n.03'), Synset('pace.n.03'), Synset('pace.n.04'), Synset('tempo.n.02'), Synset('yard.n.01'), Synset('pace.v.01'), Synset('pace.v.02'), Synset('pace.v.03'), Synset('pace.v.04')]
[Synset('besides.r.02')]
[Synset('giant.n.01'), Synset('colossus.n.02'), Synset('giant.n.03'), Synset('giant.n.04'), Synset('giant.n.05'), Synset('giant.n.06'), Synset('giant_star.n.01'), Synset('elephantine.s.01')]
[Synset('point.n.01'), Synset('point.n.02'), Synset('point.n.03'), Synset('detail.n.01'), Synset('degree.n.02'), Synset('point.n.06'), Synset('point.n.07'), Synset('point.n.08'), Synset('point.n.09'), Synset('point.n.10'), Synset('point.n.11'), Synset('item.n.01'), Synset('point.n.13'), Synset('point.n.14'), Synset('point

[Synset('high.n.01'), Synset('high.n.02'), Synset('high.n.03'), Synset('high.n.04'), Synset('high.n.05'), Synset('senior_high_school.n.01'), Synset('high_gear.n.01'), Synset('high.a.01'), Synset('high.a.02'), Synset('eminent.s.01'), Synset('high.a.04'), Synset('high.s.05'), Synset('gamey.s.02'), Synset('high.s.07'), Synset('high.r.01'), Synset('high.r.02'), Synset('high.r.03'), Synset('high.r.04')]
[Synset('gross.n.02'), Synset('tax_income.n.01')]
[Synset('cash.n.01'), Synset('cash.n.02'), Synset('cash.n.03'), Synset('cash.v.01')]
[Synset('poor_people.n.01'), Synset('hapless.s.01'), Synset('poor.a.02'), Synset('poor.a.03'), Synset('poor.a.04'), Synset('inadequate.s.02'), Synset('poor.s.06')]
[Synset('well.n.01'), Synset('well.n.02'), Synset('well.n.03'), Synset('well.n.04'), Synset('well.n.05'), Synset('well.v.01'), Synset('well.a.01'), Synset('good.s.13'), Synset('well.s.03'), Synset('well.r.01'), Synset('well.r.02'), Synset('well.r.03'), Synset('well.r.04'), Synset('well.r.05'), Syns

...

[Synset('supporter.n.01'), Synset('patron.n.03'), Synset('assistant.n.01'), Synset('garter.n.01'), Synset('athletic_supporter.n.01')]
[Synset('jimmy.n.01'), Synset('pry.v.01')]
[Synset('carter.n.01'), Synset('carter.n.02'), Synset('carter.n.03')]
[Synset('walter.n.01')]
[]
[Synset('supporter.n.01'), Synset('patron.n.03'), Synset('assistant.n.01'), Synset('garter.n.01'), Synset('athletic_supporter.n.01')]
[]
[Synset('doctor_of_optometry.n.01'), Synset('oculus_dexter.n.01')]
[]
[Synset('extremist.n.01')]
[Synset('god.n.01'), Synset('deity.n.01'), Synset('god.n.03'), Synset('idol.n.01')]
[]
[Synset('god.n.01'), Synset('deity.n.01'), Synset('god.n.03'), Synset('idol.n.01')]
[Synset('enforce.v.01'), Synset('enforce.v.02')]
[]
[]
[]
[]
[Synset('automaton.n.02')]
[]
[]
[]
[Synset('argon.n.01'), Synset('are.n.01'), Synset('arkansas.n.01')]
[]
[Synset('war.n.01'), Synset('war.n.02'), Synset('war.n.03'), Synset('war.n.04'), Synset('war.v.01')]
[Synset('ball.n.08'), Synset('swelling.n.01'), Synse

[Synset('lull.n.02'), Synset('tranquillity.n.01'), Synset('silence.n.02'), Synset('repose.n.03'), Synset('quieten.v.01'), Synset('calm.v.01'), Synset('quiet.a.01'), Synset('quiet.a.02'), Synset('quiet.s.03'), Synset('hushed.s.01'), Synset('placid.s.01'), Synset('quiet.a.06'), Synset('quietly.r.03')]
[Synset('rhenium.n.01'), Synset('ra.n.02'), Synset('re.n.03')]
[]
[Synset('avant-garde.n.01'), Synset('vanguard.n.01'), Synset('van.n.03'), Synset('van.n.04'), Synset('van.n.05')]
[Synset('delaware.n.04')]
[]
[Synset('agenda.n.01'), Synset('agenda.n.02')]
[]
[Synset('oto.n.01'), Synset('oto.n.02')]
[Synset('ear.n.01'), Synset('ear.n.02'), Synset('auricle.n.02'), Synset('ear.n.04'), Synset('ear.n.05')]
[]
[]
[Synset('great.n.01'), Synset('great.s.01'), Synset('great.s.02'), Synset('great.s.03'), Synset('bang-up.s.01'), Synset('capital.s.03'), Synset('big.s.13')]
[]
[]
[]
[Synset('ore.n.01'), Synset('ore.n.02')]
[]
[Synset('presence.n.01'), Synset('presence.n.02'), Synset('presence.n.03'), Sy

[Synset('strong.a.01'), Synset('strong.s.02'), Synset('potent.s.02'), Synset('potent.a.03'), Synset('impregnable.s.01'), Synset('solid.s.07'), Synset('strong.s.07'), Synset('hard.s.10'), Synset('strong.s.09'), Synset('firm.s.03')]
[Synset('funny_story.n.01'), Synset('amusing.s.02'), Synset('curious.s.01'), Synset('fishy.s.02'), Synset('funny.s.04')]
[Synset('girl.n.01'), Synset('miss.n.02'), Synset('miss.n.03'), Synset('miss.v.01'), Synset('miss.v.02'), Synset('miss.v.03'), Synset('neglect.v.01'), Synset('miss.v.05'), Synset('miss.v.06'), Synset('miss.v.07'), Synset('miss.v.08'), Synset('miss.v.09')]
[Synset('scene.n.01'), Synset('scene.n.02'), Synset('view.n.02'), Synset('scene.n.04'), Synset('picture.n.04'), Synset('scene.n.06'), Synset('fit.n.01'), Synset('scene.n.08'), Synset('setting.n.01'), Synset('scenery.n.01')]
[]
[]
[]
[]
[Synset('roentgen.n.01'), Synset('gas_constant.n.01'), Synset('r.n.03'), Synset('radius.n.01')]
[]
[]
[Synset('book.n.01'), Synset('book.n.02'), Synset('rec

[Synset('grey.n.01'), Synset('grey.n.02'), Synset('grey.n.03'), Synset('grey.n.04'), Synset('gray.n.01'), Synset('grey.n.06'), Synset('grey.n.07'), Synset('grey.v.01'), Synset('grey.v.02'), Synset('grey.s.01'), Synset('grey.s.02'), Synset('grey.s.03'), Synset('grey.s.04')]
[Synset('rifle.n.01'), Synset('plunder.v.03'), Synset('rifle.v.02')]
[]
[]
[]
[Synset('food.n.01'), Synset('food.n.02'), Synset('food.n.03')]
[Synset('transportation_system.n.01'), Synset('transportation.n.02'), Synset('fare.n.02'), Synset('department_of_transportation.n.01'), Synset('transportation.n.05'), Synset('exile.n.03')]
[Synset('average.n.01'), Synset('average.n.02'), Synset('average.n.03'), Synset('average.v.01'), Synset('average.v.02'), Synset('average.v.03'), Synset('average.s.01'), Synset('average.s.02'), Synset('average.s.03'), Synset('average.s.04'), Synset('modal.s.01'), Synset('median.s.01')]
[]
[]
[]
[Synset('fifty.n.01'), Synset('fifty.s.01')]
[]
[]
[Synset('astor.n.01'), Synset('astor.n.02')]
[Syn

[Synset('deny.v.01'), Synset('deny.v.02'), Synset('deny.v.03'), Synset('deny.v.04'), Synset('deny.v.05'), Synset('traverse.v.03'), Synset('deny.v.07')]
[Synset('sad.a.01'), Synset('sad.s.02'), Synset('deplorable.s.01')]
[Synset('disturbance.n.02'), Synset('upset.n.02'), Synset('disorder.n.01'), Synset('upset.n.04'), Synset('upset.n.05'), Synset('overturn.n.02'), Synset('upset.v.01'), Synset('upset.v.02'), Synset('disturb.v.01'), Synset('overturn.v.02'), Synset('swage.v.01'), Synset('upset.v.06'), Synset('disquieted.s.01'), Synset('broken.s.08'), Synset('upset.s.03'), Synset('upset.s.04'), Synset('overturned.s.01')]
[Synset('angry.a.01'), Synset('angry.s.02'), Synset('angry.s.03')]
[]
[]
[]
[]
[Synset('piece.n.01'), Synset('piece.n.02'), Synset('part.n.03'), Synset('musical_composition.n.01'), Synset('piece.n.05'), Synset('piece.n.06'), Synset('firearm.n.01'), Synset('piece.n.08'), Synset('piece.n.09'), Synset('objet_d'art.n.01'), Synset('while.n.01'), Synset('slice.n.01'), Synset('man.

In [14]:
Average_data= ((1805/3206)*100)
print (Average_data)

56.30068621334997
